# Heat Pump Model

In this tutorial we want to create a prosumer model consisting of a heat pump, heater and a storage. But as a first step, the heat demand of our building model needs to be calculated.

## Heat Demand Calculation

In pandaprosumer, single family house models are implemented. Thermal inertia of buildings is not yet considered. The building types are based on the classification by the Institute for Housing and Environment (IWU). More information on the building types can be found here: https://episcope.eu/welcome/ and here: https://webtool.building-typology.eu/#bm

For our example, we choose the building type "k" with no additional modernization.

First, some imports:

In [1]:
import os,sys
sys.path.insert(1, r"C:\Users\e2n037\git\pandaprosumer")

import numpy as np
import pandas as pd
from pandapower.timeseries.data_sources.frame_data import DFData
from pandaprosumer.controller.heat_demand import SpaceHeatingApartmentControl, SpaceHeatingElementControl
from pandaprosumer.controller.heat_demand.data_model import SpaceHeatingApartment, SpaceHeatingElement
from pandaprosumer.controller.mapping import Mapping
from pandaprosumer.create import define_space_heating_element, create_empty_prosumer_container, \
    define_space_heating_apartment, define_location, define_period
from pandaprosumer.run_time_series import run_timeseries


In this example, we create a heat demand for a temperature time-series for 1st Jan 2020, with a 15min resolution.

In [2]:
start = '2020-01-01 00:00:00'
end = '2020-01-01 23:59:59'
resol = 900
dur = pd.date_range(start, end, freq='%ss' % resol)

temperature = pd.DataFrame(np.array([3]*24+[0]*24+[-10]*36+[-5]*12), index=dur, columns=['t_amb_c'])
temperature.plot()

Similarly to pandapower and pandapipes, the input for a time-series calculation has to be converted to the DFData format, which we imported before.

In [3]:
data_source = DFData(temperature)
data_source

Then we define a function which describes the geometry of our building, according to the standardized catalogue of IWU:

In [4]:
def type_k_boundaries(prosumer, variant='var1'):
    el = []
    el += [define_space_heating_element(prosumer, variant, 132, 'roof_1', 'k', 0, 0)]
    el += [define_space_heating_element(prosumer, variant, 108, 'floor_2', 'k', 0, 0)]
    el += [define_space_heating_element(prosumer, variant, 3, 'door', 'k', 0, 90)]
    el += [define_space_heating_element(prosumer, variant, 57, 'wall_1', 'k', 0, 90)]
    el += [define_space_heating_element(prosumer, variant, 57, 'wall_1', 'k', 90, 90)]
    el += [define_space_heating_element(prosumer, variant, 57, 'wall_1', 'k', 180, 90)]
    el += [define_space_heating_element(prosumer, variant, 57, 'wall_1', 'k', 270, 90)]
    el += [define_space_heating_element(prosumer, variant, 8.4, 'window_1', 'k', 0, 0)]
    el += [define_space_heating_element(prosumer, variant, 8.4, 'window_1', 'k', 0, 90)]
    el += [define_space_heating_element(prosumer, variant, 8.4, 'window_1', 'k', 90, 90)]
    el += [define_space_heating_element(prosumer, variant, 8.4, 'window_1', 'k', 180, 90)]
    el += [define_space_heating_element(prosumer, variant, 8.4, 'window_1', 'k', 270, 90)]
    ap = [define_space_heating_apartment(prosumer, 3, 186.8 * 2.5, 186.8)]
    return el, ap


Element describes the roof, walls, windows and the floor. The variable ap sums it all up and defines the volume of the apartment. The variant 'var1' describes the modernization rate "no modernization".

In pandaprosumer for creating a prosumer container, the following 3 steps always need to be taken: create the empty prosumer container, define a location, and define a period.
In this example, we create a prosumer container with the name 'space_heating', for the location Kassel (Hessen, Germany) (latitude, longitude, elevation: 51.3128, 9.481544, 0), and a period (time-axis for the simulation later) with a resolution of 900s=15min, for a 3-day period in 2020. Timezone and a name for the period also are given.

In [5]:
location_long=9.481544
location_lat=51.312801
elevation=0
location_name='kassel'
state='HE'
country='Germany'

In [6]:
timezone='utc'
period_name='test'

In [7]:
prosumer = create_empty_prosumer_container('space_heating')
loc = define_location(prosumer, location_long, location_lat, elevation, state, country, location_name)
per = define_period(prosumer, resol, start, end, timezone, period_name)

In [8]:
prosumer

After the general container is created, we need to add our apartment and element models to the prosumer container. We define the following function:

In [9]:
def space_heating_dataclass(period_index, location_index, profile_name_t_amb_c,
                            space_heating_element, space_heating_apartment):
    el_map = Mapping('space_heating_element', space_heating_element)

    el = SpaceHeatingElement(
        element_index=space_heating_element,
        profile_name_t_amb_c=profile_name_t_amb_c,
        period_index=period_index,
        location_index=location_index)

    ap = SpaceHeatingApartment(
        element_index=space_heating_apartment,
        profile_name_t_amb_c=profile_name_t_amb_c,
        assigned_space_heating_element=[[el_map]],
        period_index=period_index,
        location_index=location_index)
    return el, ap

This function creates the dataclasses according to the given templates in pandaprosumer for the elements and the apartment, which we defined with the function "type_k_boundaries" before. Now, let's create the dataclasses

In [10]:
el, ap = space_heating_dataclass(per, loc, ['t_amb_c'], *type_k_boundaries(prosumer))
prosumer

In [11]:
el


In [12]:
ap

Then we define the controllers, both for the space heating elements and apartments:

In [13]:
shec = SpaceHeatingElementControl(prosumer, el, data_source, 0, 0)
shac = SpaceHeatingApartmentControl(prosumer, ap, data_source, 0, 1)
prosumer

Inside these two classes, the space heating calculation is taking place, which we trigger by running the time-series calculation:

In [14]:
run_timeseries(prosumer, per)
prosumer.time_series.loc[0,'data_source'].df*=-1e-6

In [15]:
prosumer

In [16]:

prosumer.time_series.data_source.at[0].df.p_th_mw.plot()
temperature.plot()

## Heat Pump Simulation

First, some imports:

In [17]:
from pandapower.timeseries.data_sources.frame_data import DFData
from pandaprosumer.controller.basics import ExtGridControl
from pandaprosumer.controller.basics.data_model import ExtGrid
from pandaprosumer.controller.constant.const import ConstProsumerControl
from pandaprosumer.controller.constant.data_model import ConstProsumer
from pandaprosumer.controller.mapping import Mapping, StorageMapping
from pandaprosumer.controller.sector_coupling import HeaterControl, HeatPumpControl
from pandaprosumer.controller.sector_coupling.data_model import Heater, HeatPump
from pandaprosumer.controller.storages import EnergyStorageControl
from pandaprosumer.controller.storages.data_model import EnergyStorage
from pandaprosumer.controller.storages.energy_storage import control_strategy as control_strategy_store, \
    finalize_strategy
from pandaprosumer.create import create_empty_prosumer_container, define_location, define_period, define_heat_demand, \
    define_heater, define_ext_grid, define_heat_pump, define_energy_storage
from pandaprosumer.run_time_series import run_timeseries


Now as the heat demand is calculated, we jump to the heat pump simulation. The heat pump system consists of a heat pump (2kW), heater (5kW) and a storage (10kWh) which all cover the heat demand calculated before. The flow temperature shall remain constant as 60°C. We define a new dataframe which has the flow temperature, ambient temperature and heat demand inside and convert it into a DFData object.

In [18]:
heat_pump_power_mw=0.002
heater_power_mw=0.005
storage_e_mwh=0.01
flow_temperature=[60] * len(dur)

hd_amb_flow = pd.DataFrame(np.array([prosumer.time_series.data_source.at[0].df.p_th_mw.values, temperature.t_amb_c.values, flow_temperature]).T, index=dur,
                     columns=['p_demand_mw', 't_amb_c', 't_sink_c'])
data_source = DFData(hd_amb_flow)

prosumer = create_empty_prosumer_container('space_heating')
loc = define_location(prosumer, location_long, location_lat, elevation, state, country, location_name)
per = define_period(prosumer, resol, start, end, timezone, period_name)

In [19]:
hd_amb_flow

Similarly to the heat demand calculation, we define a dataclass function which connects our components of the heat pump system and we define a function which defines the heat pump, heater, storage and external grid.

In [20]:
def hp_heater_energy_storage_demand_boundaries(prosumer, hp_power_mw, heater_power_mw, storage_e_mwh):
    demand = define_heat_demand(prosumer, 0.15)
    store = define_energy_storage(prosumer, 0, storage_e_mwh, 50, 100, 100)
    hp = define_heat_pump(prosumer, hp_power_mw, 0)
    heater = define_heater(prosumer, heater_power_mw, 0)
    ext = define_ext_grid(prosumer, p_el_mw=0, q_el_mvar=0, p_th_mw=0)
    return [demand], [store], [hp], [heater], [ext]


def hp_heater_energy_storage_demand_dataclass(period_index, location_index,
                                              profile_name_demand, profile_name_t_amb_c,
                                              profile_name_t_source_c, profile_name_t_sink_c,
                                              demand_index, energy_storage_index, heat_pump_index, heater_index,
                                              ext_grid_index):
    demand = ConstProsumer(
        element='heat_demand',
        element_index=demand_index,
        element_variable='p_th_mw',
        profile_name=profile_name_demand,
        period_index=period_index)
    storage = EnergyStorage(
        element_variable='p_th_mw',
        element_index=energy_storage_index,
        assigned_element=[[demand]],
        period_index=period_index,
        location_index=location_index)
    mapping_storage_hp = StorageMapping(
        element='energy_storage',
        element_variable='p_th_mw',
        element_index=energy_storage_index,
        upper_limit=[100],
        lower_limit=[50])
    hp = HeatPump(
        element_variable='p_el_mw',
        element_index=heat_pump_index,
        efficiency_variable='eff_percent',
        assigned_element=[[mapping_storage_hp]],
        profile_name_t_amb_c=profile_name_t_amb_c,
        profile_name_t_sink_c=profile_name_t_sink_c,
        profile_name_t_source_c=profile_name_t_source_c,
        control_strategy=[control_strategy_store],
        finalize_strategy=[finalize_strategy],
        period_index=period_index,
        location_index=location_index)
    mapping_storage_heater = StorageMapping(
        element='energy_storage',
        element_variable='p_th_mw',
        element_index=energy_storage_index,
        upper_limit=[100],
        lower_limit=[30],
        initial_state=[False])
    heater = Heater(
        element_variable='p_el_mw',
        element_index=heater_index,
        efficiency_variable='eff_percent',
        assigned_element=[[mapping_storage_heater]],
        profile_name_t_amb_c=profile_name_t_amb_c,
        control_strategy=[control_strategy_store],
        finalize_strategy=[finalize_strategy],
        period_index=period_index,
        location_index=location_index)
    mapping_heat_demand = Mapping(
        element='heat_demand',
        element_index=demand_index)
    mapping_energy_storage = Mapping(
        element='energy_storage',
        element_index=energy_storage_index)
    mapping_heat_pump = Mapping(
        element='heat_pump',
        element_index=heat_pump_index)
    mapping_heater = Mapping(
        element='heater',
        element_index=heater_index)
    ext = ExtGrid(
        element='ext_grid',
        element_index=ext_grid_index,
        period_index=period_index,
        finalize_time_series_only=False,
        assigned_element=[[mapping_heat_demand, mapping_energy_storage, mapping_heat_pump, mapping_heater]])
    return demand, storage, hp, heater, ext

In [21]:
profile_name_demand=['p_demand_mw']
profile_name_t_amb_c=['t_amb_c']
profile_name_t_source_c=['t_amb_c']
profile_name_t_sink_c=['t_sink_c']

In [22]:
demand, store, hp, heater, ext_grid = hp_heater_energy_storage_demand_dataclass(
    per, loc, profile_name_demand, profile_name_t_amb_c, profile_name_t_source_c, profile_name_t_sink_c,
    *hp_heater_energy_storage_demand_boundaries(prosumer, heat_pump_power_mw, heater_power_mw, storage_e_mwh))

Adding the controllers:

In [23]:
def couple_hp_heater_energy_storage_demand(prosumer, demand_object, energy_storage_object,
                                           heat_pump_object, heater_object, ext_grid_object, data_source,
                                           order=[0, 1, 2, 3, 4], level=[0, 0, 0, 0, 0]):
    ConstProsumerControl(prosumer, demand_object, data_source, order[0], level[0])
    EnergyStorageControl(prosumer, energy_storage_object, order[1], level[1])
    HeatPumpControl(prosumer, heat_pump_object, data_source, order[2], level[2])
    HeaterControl(prosumer, heater_object, data_source, order[3], level[3])
    ExtGridControl(prosumer, ext_grid_object, order[4], level[4])


In [24]:
couple_hp_heater_energy_storage_demand(prosumer, demand, store, hp, heater, ext_grid, data_source)

In [25]:
run_timeseries(prosumer, per, True)

In [26]:
prosumer

In [27]:
prosumer.controller

In [28]:
prosumer.heat_demand

In [29]:
prosumer.heat_pump

In [30]:
prosumer.energy_storage

In [31]:
prosumer.heater

In [32]:
prosumer.ext_grid

In [33]:
prosumer.time_series

In [34]:
prosumer.time_series.loc[1,'data_source'].df.plot()


In [35]:
prosumer.time_series.loc[2,'data_source'].df.plot()


In [36]:
prosumer.time_series.loc[3,'data_source'].df.plot()

In [37]:
prosumer.time_series.loc[4,'data_source'].df.plot()

In [38]:
df_heat_pump_profiles = pd.DataFrame(index=dur)
df_heat_pump_profiles['p_el_mw'] = prosumer.time_series.loc[2,'data_source'].df.p_el_mw+prosumer.time_series.loc[3,'data_source'].df.p_el_mw

In [39]:
df_heat_pump_profiles.to_csv('hp_profile_workshop.csv')